# Capstone Coursera-IBM Python for Data-Science

This notebook will be used to be peer-reviewed in the module 3 of the capstone IBM.

## Module 3

Import some libraries

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

### Part 1

#### Get data from Wikipedia

In [2]:
data=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#### Use BeautifulSoup to parse data

In [3]:
data = BeautifulSoup(data, 'html.parser')
zc=[] #zipcode
bl=[] #borough list
nl=[] #neighborhood

In [4]:
for r in data.find('table').find_all('tr'):
    c=r.find_all('td')
    if(len(c)>0):
        #print(c[0].text)
        zc.append(c[0].text.rstrip('\n'))
        bl.append(c[1].text.rstrip('\n'))
        nl.append(c[2].text.rstrip('\n'))

In [5]:
t_df=pd.DataFrame({'PostalCode':zc,
                   'Borough':bl,
                   'Neighborhood':nl})

In [6]:
t_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [7]:
t_df.PostalCode.duplicated().any()

False

There's no duplicate value in PostalCode.

#### Drop Borough='Not assigned' lines and correct Neighborhood='Not assigned'

In [8]:
t_df=t_df[t_df.Borough!="Not assigned"].reset_index(drop=True)
for index, r in t_df.iterrows():
    if r["Neighborhood"] == "Not assigned":
        r["Neighborhood"] = r["Borough"]

In [9]:
t_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


It seems that something changes on wikipedia as the neiborhood seems to be separed by a ' / '

#### Replace '/' by ','

In [10]:
for r in t_df.index:
    t_df.Neighborhood[r]=t_df.Neighborhood[r].replace(' / ',', ')

In [11]:
t_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Finally, last question of part 1 : shape of the DataFrame

In [12]:
t_df.shape

(103, 3)

### Part 2

#### Load the coordinates from the CSV given by Coursera

In [13]:
coord=pd.read_csv("Geospatial_Coordinates.csv")
coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
coord['Postal Code'].duplicated().any()

False

There's no duplicate value in Postal Code.
The column is entitled "PostalCode" in t_df, the Toronto DataFrame from Wikipedia and "Postal Code" in coord, the DataFrame form Coursera. It should be made consistent before to merge.

In [15]:
coord.rename(columns={"Postal Code":"PostalCode"},inplace=True)
coord.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merge the two tables

In [16]:
t_df_full=t_df.merge(coord, on="PostalCode", how="left")

In [17]:
t_df_full.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [18]:
t_df_full

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road , Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


#### Give the same DF as in the Coursera Assignments

In [18]:
t_df_pr=pd.DataFrame(columns=t_df_full.columns)
t_list=["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]
for pc in t_list:
    t_df_pr=t_df_pr.append(t_df_full[t_df_full['PostalCode'] == pc],ignore_index=True)

t_df_pr

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


### Part 3

#### Use Geopy and folium to create a map of Toronto

In [19]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
loc = geolocator.geocode(address)
lat = loc.latitude
long = loc.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address,lat, long))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [20]:
map_toronto=folium.Map(location=[lat,long],zoom_start=9)
map_toronto

#### Add Borough containing "Toronto" and Neighborhood

In [21]:
bt=list(set([t_df_full.Borough[i] for i in range(0,t_df_full.Borough.shape[0])])) #df->list->set(to avoid duplicate)->list
bt=[bt[i] for i in range(0,len(bt)) if "toronto" in bt[i].lower()] #list->list with only "toronto-name"
bt

['West Toronto', 'East Toronto', 'Central Toronto', 'Downtown Toronto']

In [22]:
tt_df=t_df_full[t_df_full.Borough.isin(bt)].reset_index(drop=True)
tt_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [112]:
map_toronto = folium.Map(location=[lat, long], zoom_start=12)

for la, lo, borough, neighborhood in zip(tt_df['Latitude'], tt_df['Longitude'], tt_df['Borough'], tt_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [la, lo],
        radius=5,
        popup=label,
        color='blue').add_to(map_toronto)  
    
map_toronto 

#### Let's go on Foursquare

In [113]:
fs_api=pd.read_csv('foursquare.csv',header=None) #Credentials

In [114]:
CLIENT_ID = fs_api[0][0] # your Foursquare ID
CLIENT_SECRET = fs_api[1][0] # your Foursquare Secret
VERSION = fs_api[2][0] # Foursquare API version

Top 100 in the 500m around the (lat,long) of each borough

In [115]:
limit=100
r=500

venues=[]

for la, lo, post, borough, neighborhood in zip(tt_df['Latitude'], tt_df['Longitude'], tt_df['PostalCode'], 
                                               tt_df['Borough'], tt_df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID, CLIENT_SECRET, VERSION,
        la, lo,
        r, limit)
    
    rget = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in rget:
        venues.append((
            post, 
            borough,
            neighborhood,
            la, 
            lo, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [116]:
v_df=pd.DataFrame(venues)
v_df.columns=['PostalCode','Borough','Neighborhood','Latitude','Longitude',
             'VenueName','VenueLatitude','VenuesLongitude','VenueCategory']

In [117]:
v_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenuesLongitude,VenueCategory
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [118]:
v_df.to_csv('venus_toronto.csv',index=False) #To be able to continue without request Foursquare during dev

In [23]:
v_df=pd.read_csv('venus_toronto.csv')

In [24]:
v_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenuesLongitude,VenueCategory
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


#### How many venues were returned for each Postal Code

In [25]:
v_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,Latitude,Longitude,VenueName,VenueLatitude,VenuesLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,4,4,4,4,4,4
M4K,East Toronto,"The Danforth West, Riverdale",42,42,42,42,42,42
M4L,East Toronto,"India Bazaar, The Beaches West",18,18,18,18,18,18
M4M,East Toronto,Studio District,40,40,40,40,40,40
M4N,Central Toronto,Lawrence Park,3,3,3,3,3,3
M4P,Central Toronto,Davisville North,8,8,8,8,8,8
M4R,Central Toronto,North Toronto West,24,24,24,24,24,24
M4S,Central Toronto,Davisville,32,32,32,32,32,32
M4T,Central Toronto,"Moore Park, Summerhill East",2,2,2,2,2,2


In [26]:
print('Number of categories: {}'.format(len(v_df['VenueCategory'].unique())))

Number of categories: 227


#### Analyze each Area

##### Code from NY example :
```python

# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()
```

In [27]:
t_onehot = pd.get_dummies(v_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
t_onehot['PostalCode'] = v_df['PostalCode'] 
t_onehot['Borough'] = v_df['Borough'] 
t_onehot['Neighborhoods'] = v_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(t_onehot.columns[-3:]) + list(t_onehot.columns[:-3])
t_onehot = t_onehot[fixed_columns]

print(t_onehot.shape)
t_onehot.head()

(1635, 230)


,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

```python
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped
```

In [28]:
t_grouped = t_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()
t_grouped

,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.00000,0.25000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,M4K,East Toronto,"The Danforth West, Riverdale",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.00000,0.02381,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810
2,M4L,East Toronto,"India Bazaar, The Beaches West",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,M4M,East Toronto,Studio District,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.025000,0.000000,0.025000
4,M4N,Central Toronto,Lawrence Park,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,M4P,Central Toronto,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,M4R,Central Toronto,North Toronto West,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667
7,M4S,Central Toronto,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.03125,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,M4T,Central Toronto,"Moore Park, Summerhill East",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000


In [29]:
print("DataFrames'size :\n- Onehot :",t_onehot.shape,"\n- Grouped :",t_grouped.shape)

DataFrames'size :
- Onehot : (1635, 230) 
- Grouped : (39, 230)


#### Let's put each neighborhood along with the top 10 most common venues in a DF

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues // code from NY example mainly
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = t_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = t_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = t_grouped['Neighborhoods']

for ind in np.arange(t_grouped.shape[0]):
    row_categories = t_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(39, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Health Food Store,Neighborhood,Pub,Trail,Yoga Studio,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center
1,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Bookstore,Ice Cream Shop,Pub,Pizza Place,Lounge
2,M4L,East Toronto,"India Bazaar, The Beaches West",Fast Food Restaurant,Park,Sushi Restaurant,Pet Store,Movie Theater,Pub,Restaurant,Burrito Place,Liquor Store,Sandwich Place
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Gastropub,Brewery,Bakery,American Restaurant,Neighborhood,Sandwich Place,Cheese Shop,Pet Store
4,M4N,Central Toronto,Lawrence Park,Park,Swim School,Bus Line,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
5,M4P,Central Toronto,Davisville North,Convenience Store,Sandwich Place,Park,Gym,Hotel,Breakfast Spot,Food & Drink Shop,Department Store,Diner,Discount Store
6,M4R,Central Toronto,North Toronto West,Clothing Store,Coffee Shop,Yoga Studio,Salon / Barbershop,Restaurant,Rental Car Location,Café,Chinese Restaurant,Pet Store,Miscellaneous Shop
7,M4S,Central Toronto,Davisville,Sandwich Place,Dessert Shop,Pizza Place,Gym,Italian Restaurant,Café,Sushi Restaurant,Coffee Shop,Greek Restaurant,Pharmacy
8,M4T,Central Toronto,"Moore Park, Summerhill East",Playground,Tennis Court,Yoga Studio,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",Pub,Coffee Shop,Fried Chicken Joint,Light Rail Station,Liquor Store,Restaurant,Sports Bar,Bank,Supermarket,Bagel Shop


### Cluster Areas

Run *k*-means to cluster the neighborhood into 5 clusters.

In [34]:
# set number of clusters
kclusters = 8

t_grouped_clustering = t_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(t_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([3, 0, 0, 0, 5, 7, 0, 0, 4, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [35]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
t_merged = tt_df.copy()

# add clustering labels
t_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
t_merged = t_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

print(t_merged.shape)
t_merged.sort_values(["Cluster Labels"], inplace=True)
t_merged # check the column between Longitude and Venues

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,M5N,Central Toronto,Roselawn,43.711695,-79.416936,0,Garden,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
20,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Convenience Store,Sandwich Place,Park,Gym,Hotel,Breakfast Spot,Food & Drink Shop,Department Store,Diner,Discount Store
21,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307,0,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Yoga Studio,Department Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0,Sandwich Place,Café,Coffee Shop,Park,Cosmetics Shop,Liquor Store,Burger Joint,Indian Restaurant,Middle Eastern Restaurant,Pub
25,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325,0,Gift Shop,Cuban Restaurant,Eastern European Restaurant,Breakfast Spot,Italian Restaurant,Dog Run,Bar,Dessert Shop,Restaurant,Movie Theater
26,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Sandwich Place,Dessert Shop,Pizza Place,Gym,Italian Restaurant,Café,Sushi Restaurant,Coffee Shop,Greek Restaurant,Pharmacy
27,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,0,Café,Yoga Studio,Bakery,Restaurant,Bookstore,Italian Restaurant,Bar,Japanese Restaurant,Sushi Restaurant,Coffee Shop
28,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450,0,Café,Sushi Restaurant,Pizza Place,Coffee Shop,Italian Restaurant,Pub,Bookstore,Juice Bar,Restaurant,Burrito Place
29,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,0,Playground,Tennis Court,Yoga Studio,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
30,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,0,Café,Coffee Shop,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Mexican Restaurant,Grocery Store,Dessert Shop,Dumpling Restaurant,Chinese Restaurant


Finally, let's visualize the resulting clusters

In [36]:
# create map
map_clusters = folium.Map(location=[lat, long], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(t_merged['Latitude'], t_merged['Longitude'], t_merged['PostalCode'], 
                                             t_merged['Borough'], t_merged['Neighborhood'], 
                                             t_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Clusters

#### Cluster 0

In [130]:
t_merged.loc[t_merged['Cluster Labels'] == 0, t_merged.columns[[1] + list(range(5, t_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,0,Garden,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
21,Central Toronto,0,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Yoga Studio,Department Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
23,Central Toronto,0,Clothing Store,Coffee Shop,Yoga Studio,Salon / Barbershop,Restaurant,Rental Car Location,Café,Chinese Restaurant,Pet Store,Miscellaneous Shop
24,Central Toronto,0,Sandwich Place,Café,Coffee Shop,Park,Cosmetics Shop,Liquor Store,Burger Joint,Indian Restaurant,Middle Eastern Restaurant,Pub
25,West Toronto,0,Gift Shop,Cuban Restaurant,Eastern European Restaurant,Breakfast Spot,Italian Restaurant,Dog Run,Bar,Dessert Shop,Restaurant,Movie Theater
26,Central Toronto,0,Sandwich Place,Dessert Shop,Pizza Place,Gym,Italian Restaurant,Café,Sushi Restaurant,Coffee Shop,Greek Restaurant,Pharmacy
27,Downtown Toronto,0,Café,Yoga Studio,Bakery,Restaurant,Bookstore,Italian Restaurant,Bar,Japanese Restaurant,Sushi Restaurant,Coffee Shop
28,West Toronto,0,Café,Sushi Restaurant,Pizza Place,Coffee Shop,Italian Restaurant,Pub,Bookstore,Juice Bar,Restaurant,Burrito Place
29,Central Toronto,0,Playground,Tennis Court,Yoga Studio,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
30,Downtown Toronto,0,Café,Coffee Shop,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Mexican Restaurant,Grocery Store,Dessert Shop,Dumpling Restaurant,Chinese Restaurant


#### Cluster 1

In [131]:
t_merged.loc[t_merged['Cluster Labels'] == 1, t_merged.columns[[1] + list(range(5, t_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,West Toronto,1,Mexican Restaurant,Bar,Grocery Store,Café,Thai Restaurant,Fast Food Restaurant,Diner,Cajun / Creole Restaurant,Speakeasy,Bakery


#### Cluster 2

In [132]:
t_merged.loc[t_merged['Cluster Labels'] == 2, t_merged.columns[[1] + list(range(5, t_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,2,Health Food Store,Neighborhood,Pub,Trail,Yoga Studio,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center
10,Downtown Toronto,2,Coffee Shop,Aquarium,Restaurant,Italian Restaurant,Hotel,Café,Scenic Lookout,Fried Chicken Joint,Sporting Goods Shop,Brewery


#### Cluster 3

In [133]:
t_merged.loc[t_merged['Cluster Labels'] == 3, t_merged.columns[[1] + list(range(5, t_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Gym,Deli / Bodega,Hotel,Thai Restaurant,Asian Restaurant,Breakfast Spot,Gym / Fitness Center


#### Cluster 4

In [134]:
t_merged.loc[t_merged['Cluster Labels'] == 4, t_merged.columns[[1] + list(range(5, t_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,4,Coffee Shop,Park,Bakery,Pub,Mexican Restaurant,Breakfast Spot,Café,Theater,Shoe Store,Restaurant


### Conclusion

- Most of the neighborhood are classified in cluster 0 and mainly the venues are business such as resturants or supermarkets, but also park.
- Cluster 1, 2, 3 and 4 are a lot smaller.